<a href="https://colab.research.google.com/github/lbrandoli/ProgramacionConcurrente/blob/main/tp2/openMP/TP_2_openMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.1. Ejercicio 1 - Hola Mundo estilo Programación Concurrente

## a)Identifique los 3 componentes de openMP en el ejercicio 1.

1. Directivas al compilar: son las instrucciones que se utilizan para indicar al compilador que seccion de codigo debe ser paralelizada, en este caso, se utiliza "#pragma omp parallel" para indicar que la region de codigo que sigue debe ser ejecutada en paralelo.

2. Rutinas de tiempo de ejecucion: son las funciones que se utilizan para controlar el comportamiento de los hilos en tiempo de ejecucion. En el codigo se utiiza las funciones "omp_get_thread_num()", "omp_get_num_threads()", "omp_get_num_procs()" y "omp_set_num_threads()" para obtener informacion sobre los hilos y procesadores disponibles.

3. Variables de entorno: son las variables que se utilizan para controlar el comportamiento del sistema en tiempo de ejecucion, en el codigo proporcionado, vemos "OMP_NUM_THREADS" para establecer el numero maximo de hilos que se pueden utilizar.

## b)Realice pruebas modificando la cantidad de hilos (1,3,7). Que conclusiones pudo determinar?.

Si se utiliza un solo hilo, el programa se va a ejecutar de forma secuencial y no habra paralelismo.
Si se utilizan varios hilos el programas se ejecutara en paralelo y disminuira el tiempo de ejecucion. Sin embargo, si se utilizan muchos hilos, el tiempo de ejecucion podria aumentar debido a la sobrecarga asociada con la creacion y administracion de hilos.
Lo mas optimo es utilizar un numero de hilos similar al numero de nucleos disponibles en el procesador.

## c)¿Cual es la diferencia entre las formas de asignacion static y dynamic en la clausula schedule?¿Que sucede si utiliza dynamic en el codigo?

La asignacion **static** significa que las iteraciones se asignan a los hilos de manera estatica, es decir, se decide al principio que hilo hara que iteraciones. Por otro lado, la asignacion **dynamic** significa que cada hilo trabaja en un bloque de iteraciones y luego toma el siguiente bloque que no ha sido trabajado por ningun hilo.
Si se utiliza la asignacion **dynamic** en el codigo proporcionado, cada hilo trabajara en un bloque de iteraciones y luego tomara el siguiente bloque que no haya sido trabajado por ningun hilo. Esto puede llevar a un mejor equlibrio de carga si las iteraciones tiene diferentes tiempos de ejecucion, pero tambien puede aumentar la sobrecarga debido a la comunicacion entre los hilos.



## d)En el for:¿Que sucede con el valor de j, si se parametriza como lastprivate?

La clausula **lastprivate** se utiliza para asegurar que la variable privada de un bucle tenga el valor de la ultima iteracion despues de que se complete el bucle. Si se utiliza la clausula **lastprivate** en el codigo proporcionado, el valor final de j despues del bucle sera el valor de j en la ultima iteracion del bucle.

# 1.2. Ejercicio 2 - Codigo faltante:

In [ ]:
%%writefile total_ventas.cpp

#include <omp.h>
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define SEED 4
#define LIMIT_INF 0
#define LIMIT_SUP 100
#define DAYS 15

int main(int argc, char *argv[])
{
    if (argv[1])
    {
        int count_suc = atoi(argv[1]);
        int ventas_x_sucursal[count_suc][DAYS];
        int total = 0, sum;

        srand(SEED);
        for (int i = 0; i < count_suc; i++)
        {
            for (int j = 0; j < DAYS; j++)
            {
                ventas_x_sucursal[i][j] = (rand() % (LIMIT_SUP - LIMIT_INF + 1)) + LIMIT_INF;
            }
        }

        for (int i = 0; i < count_suc; i++)
        {
            for (int j = 0; j < DAYS; j++)
            {
                printf("%d \t", ventas_x_sucursal[i][j]);
            }
            printf("\n");
        }

        for (int i = 0; i < count_suc; i++)
        {
            #pragma omp parallel for reduction(+ : sum) // lastprivate(total)
            for (int j = 0; j < DAYS; j++)
            {
                #pragma omp atomic
                total += ventas_x_sucursal[i][j];
            }
        }

        printf("%d\n", total);
    }
    else
    {
        printf("Por favor, ingrese la cantidad de sucursales");
    }
}

Overwriting total_ventas.cpp


In [ ]:
!g++ -o total_ventas -fopenmp total_ventas.cpp

In [ ]:
!./total_ventas 5

78 	30 	7 	32 	71 	10 	85 	11 	85 	26 	11 	96 	70 	80 	21 	
52 	85 	65 	62 	20 	59 	97 	83 	82 	47 	78 	98 	67 	2 	39 	
47 	46 	70 	20 	78 	6 	31 	62 	18 	15 	54 	96 	10 	23 	76 	
98 	42 	26 	62 	3 	46 	20 	100 	95 	69 	12 	73 	32 	79 	75 	
72 	93 	87 	7 	12 	65 	13 	9 	93 	98 	24 	47 	94 	1 	36 	
3884


# 1.3. Ejercicio 3 - Axpy

**b) Repita la prueba (a), con ciclos 10, 100. ¿Cuál fue la diferencia?**


Cuando tengo muchos datos a procesar, aumentar la cantidad de ciclos implica una mejora en las métricas. En caso contrario, cuando tengo pocos datos a procesar, aumentar la cantidad de ciclos no genera una mejora en las métricas. Esto es debido a que estoy realizando la operación de suma vectorial N cantidad de veces (cantidad de ciclos). En conclusión, al procesar pocos datos y aumentar el número de ciclos, no hay una mejora sobre las métricas debido que el paralelismo se aplica en la función axpy() y no en el ciclo for más externo, el cual es el que se ve afectado por la variable "ciclos".






**c) ¿Cómo varía el SpeedUp a medida que aumenta o disminuye la cantidad de hilos?**

En nuestro contexto de ejecución tenemos solamente dos núcleos. Por lo tanto, a la hora de implementar una mayor cantidad de hilos se genera un overhead producto de los cambios de contexto entre hilos. En conclusión, para una misma cantidad de elementos y ciclos, el SpeedUp empeora (disminuye) con mayor número de hilos que núcleos de procesamiento, debido al overhead generado por los cambios de contexto.

**d) ¿Qué sucede con la eficiencia a medida que aumentan la cantidad_N? ¿Por qué no llega al valor ideal?**

A medida que aumentan los elementos, se genera una mejora de la eficiencia debido que los algoritmos paralelos están diseñados para resolver la problemática del procesamiento de grandes volúmenes de datos. Por lo tanto, con pocos datos no se va a observar una mejora en comparación de la programación secuencial.

No se llega al valor ideal de eficiencia debido que la creación y administración de hilos genera un overhead adicional al sistema, el cual no se contempla en la programación secuencial ya que es propio de la programación parelela. También puede ser por la sincronización o comunicación que tiene que existir entre los núcleos para acceder a la memoria.
